In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Conv2D,  MaxPool2D, Flatten, GlobalAveragePooling2D,  BatchNormalization, Layer, Add
from keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import utils
from keras.models import Model

In [8]:
model = ResNet50(include_top = True, weights = 'imagenet')
model.summary()
model.save('model.h5')

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

# قسمت اول

<div style="direction:rtl">
در این قسمت مدل resnet50 استفاده شده را لود می کنیم و با استفاده از آن، مدل را روی یک دیتاست از نوع cifar10 آموزش می دهیم.

در قسمت زیر لایه ورودی و خروجی تغییر کرده تا متناسب با داده های جدید و نیز لیبل های خروجی 10 تایی دیتاست شود.

همچنین تنها لایه آخر که fully connected است را به صورت trainable قرار داده ایم. بنابراین برای این مدل تنها وزن های لایه آخر آموزش میبیند. این کار باعث کاهش دقت مدل خواهد شد ولی در عین حال سرعت آموزش مدل را هم افزایش خواهد داد.

همانطور که مشخص است دقت مدل به تدریج در حال افزایش است ولی سرعت این افزایش زیاد نیست. 

این مدل در قسمت بعدی به عنوان teacher برای مدل قبلی مورد استفاده قرار می گیرد.

</div>

In [9]:
model_pretrained = tf.keras.models.load_model('model.h5')

model_config = model_pretrained.get_config()
model_config['layers'][0]['config']['batch_input_shape'] = (None, 32, 32, 3)
model_pretrained = model.__class__.from_config(model_config, custom_objects={})

model_pretrained.trainable = False

model_resnet50_teacher = model_pretrained.layers[-2].output
fc_layer = Dense(10, activation = "softmax")(model_resnet50_teacher)
model_resnet50_teacher = tf.keras.models.Model(inputs = model_pretrained.input, outputs = fc_layer)

model_resnet50_teacher.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

In [10]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 6s 0us/step


In [11]:
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

In [12]:
model_resnet50_teacher.compile(optimizer='adam',loss='categorical_crossentropy',
                      metrics=['accuracy'])
callback = keras.callbacks.EarlyStopping(monitor='accuracy', patience=8)
model_resnet50_teacher.fit(x_train, y_train, epochs=50, batch_size=2000, validation_data=(x_test, y_test), callbacks=[callback])

Epoch 1/50
25/25 [==============================] - 17s 230ms/step - loss: 4.7404 - accuracy: 0.1130 - val_loss: 2.6788 - val_accuracy: 0.1632
Epoch 2/50
25/25 [==============================] - 5s 212ms/step - loss: 2.4865 - accuracy: 0.1733 - val_loss: 2.2502 - val_accuracy: 0.2028
Epoch 3/50
25/25 [==============================] - 5s 192ms/step - loss: 2.1796 - accuracy: 0.2181 - val_loss: 2.1215 - val_accuracy: 0.2331
Epoch 4/50
25/25 [==============================] - 5s 212ms/step - loss: 2.1026 - accuracy: 0.2393 - val_loss: 2.0705 - val_accuracy: 0.2502
Epoch 5/50
25/25 [==============================] - 5s 192ms/step - loss: 2.0569 - accuracy: 0.2586 - val_loss: 2.0281 - val_accuracy: 0.2694
Epoch 6/50
25/25 [==============================] - 5s 192ms/step - loss: 2.0149 - accuracy: 0.2753 - val_loss: 1.9932 - val_accuracy: 0.2808
Epoch 7/50
25/25 [==============================] - 5s 213ms/step - loss: 1.9845 - accuracy: 0.2869 - val_loss: 1.9602 - val_accuracy: 0.2976
Epoch

In [13]:
!pip install image-classifiers==0.2.2
!pip install segmentation_models==1.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 KB 6.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 KB 5.7 MB/s eta 0:00:00
  Attempting uninstall: image-classifiers
    Found existing installation: image-classifiers 0.2.2
    Uninstalling image-classifiers-0.2.2:
      Successfully uninstalled image-classifiers-0.2.2


# قسمت دوم

<div style="direction:rtl">
در اینجا یک مدل resnet18 آماده لود کرده و ورودی و خروجی آن را تغییر میدهیم تا برای استفاده از دیتاس cifar10 آماده شود.

سپس این مدل را با روش knowledge distiller روی داده های جدید آموزش میدهیم.
منبع مورد استفاده برای پیاده سازی این بخش آدرس زیر است:

https://keras.io/examples/vision/knowledge_distillation/

در اینجا مقادیر alpha و temperature به صورت تجربی با آزمون و خطا با مقادیر زیر بدست آمده است:

alpha = 0.5

temperature = 1

با بررسی اعداد مختلف مشاهده شد که دقت چندان بالا نمی رود و مدل به خوبی آموزش نمیبیند.


</div>

In [14]:
from classification_models.keras import Classifiers

ResNet18, preprocess_input = Classifiers.get('resnet18')
model = ResNet18((32, 32, 3), weights='imagenet')
model.save('model2.h5')

46977688/46977688 [==============================] - 2s 0us/step


In [15]:
model_pretrained = tf.keras.models.load_model('model2.h5')

model_resnet18_student = model_pretrained.layers[-2].output
fc_layer = Dense(10, activation = "softmax")(model_resnet18_student)
model_resnet18_student = tf.keras.models.Model(inputs = model_pretrained.input, outputs = fc_layer)

model_config = model_resnet18_student.get_config()
model_config['layers'][0]['config']['batch_input_shape'] = (None, 32, 32, 3)
model_resnet18_student = model.__class__.from_config(model_config, custom_objects={})

model_resnet18_student.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 bn_data (BatchNormalization)   (None, 32, 32, 3)    9           ['data[0][0]']                   
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 38, 38, 3)   0           ['bn_data[0][0]']                
                                                                                                  
 conv0 (Conv2D)                 (None, 16, 16, 64)   9408        ['zero_padding2d[0][0]']         
                                                                                            

In [16]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            # Compute scaled distillation loss from https://arxiv.org/abs/1503.02531
            # The magnitudes of the gradients produced by the soft targets scale
            # as 1/T^2, multiply them by T^2 when using both hard and soft targets.
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [31]:
# Initialize and compile distiller
distiller = Distiller(student=model_resnet18_student, teacher=model_resnet50_teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.Accuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=keras.losses.CategoricalCrossentropy(),
    alpha=0.5,
    temperature=1,
)

callback = keras.callbacks.EarlyStopping(monitor='accuracy', patience=8)
distiller.fit(x_train, y_train, epochs=50, batch_size=2000, validation_data=(x_test, y_test), callbacks=[callback])

Epoch 1/50
25/25 [==============================] - 16s 487ms/step - accuracy: 0.0338 - student_loss: 0.0224 - distillation_loss: 2.3393 - val_accuracy: 0.0186 - val_student_loss: 2.4294
Epoch 2/50
25/25 [==============================] - 12s 471ms/step - accuracy: 0.0355 - student_loss: 0.0213 - distillation_loss: 2.3393 - val_accuracy: 0.0251 - val_student_loss: 2.5352
Epoch 3/50
25/25 [==============================] - 12s 470ms/step - accuracy: 0.0366 - student_loss: 0.0167 - distillation_loss: 2.3395 - val_accuracy: 0.0288 - val_student_loss: 2.4566
Epoch 4/50
25/25 [==============================] - 12s 468ms/step - accuracy: 0.0417 - student_loss: 0.0119 - distillation_loss: 2.3396 - val_accuracy: 0.0252 - val_student_loss: 2.6277
Epoch 5/50
25/25 [==============================] - 12s 483ms/step - accuracy: 0.0470 - student_loss: 0.0131 - distillation_loss: 2.3396 - val_accuracy: 0.0353 - val_student_loss: 2.7360
Epoch 6/50
25/25 [==============================] - 12s 468ms/ste

# قسمت سوم

<div style="direction:rtl">
در این قسمت یک مدل resnet18 را از ابتدا روی cifar10 آموزش می دهیم. همانطور که مشخص است نتایج این قسمت بسیار خوب است و به دقت های بالایی رسیده است.

در این مدل هم تمامی وزن ها قابل آموزش هستند.
</div>

In [21]:
model_pretrained = tf.keras.models.load_model('model2.h5')

model_resnet18_zero = model_pretrained.layers[-2].output
fc_layer = Dense(10, activation = "softmax")(model_resnet18_zero)
model_resnet18_zero = tf.keras.models.Model(inputs = model_pretrained.input, outputs = fc_layer)

model_config = model_resnet18_zero.get_config()
model_config['layers'][0]['config']['batch_input_shape'] = (None, 32, 32, 3)
model_resnet18_zero = model.__class__.from_config(model_config, custom_objects={})

model_resnet18_zero.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 bn_data (BatchNormalization)   (None, 32, 32, 3)    9           ['data[0][0]']                   
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 38, 38, 3)   0           ['bn_data[0][0]']                
                                                                                                  
 conv0 (Conv2D)                 (None, 16, 16, 64)   9408        ['zero_padding2d[0][0]']         
                                                                                            

In [22]:
model_resnet18_zero.compile(optimizer='adam',loss='categorical_crossentropy',
                      metrics=['accuracy'])
callback = keras.callbacks.EarlyStopping(monitor='accuracy', patience=8)
model_resnet18_zero.fit(x_train, y_train, epochs=50, batch_size=2000, validation_data=(x_test, y_test), callbacks=[callback])


Epoch 1/50
25/25 [==============================] - 10s 337ms/step - loss: 1.9269 - accuracy: 0.3570 - val_loss: 35.3019 - val_accuracy: 0.1794
Epoch 2/50
25/25 [==============================] - 8s 320ms/step - loss: 1.2535 - accuracy: 0.5459 - val_loss: 16.2973 - val_accuracy: 0.1658
Epoch 3/50
25/25 [==============================] - 8s 338ms/step - loss: 1.0075 - accuracy: 0.6374 - val_loss: 2.6088 - val_accuracy: 0.2706
Epoch 4/50
25/25 [==============================] - 8s 327ms/step - loss: 0.7776 - accuracy: 0.7225 - val_loss: 2.0201 - val_accuracy: 0.3484
Epoch 5/50
25/25 [==============================] - 8s 324ms/step - loss: 0.5599 - accuracy: 0.8006 - val_loss: 1.7506 - val_accuracy: 0.4216
Epoch 6/50
25/25 [==============================] - 8s 321ms/step - loss: 0.3996 - accuracy: 0.8595 - val_loss: 1.9884 - val_accuracy: 0.4600
Epoch 7/50
25/25 [==============================] - 8s 319ms/step - loss: 0.2860 - accuracy: 0.8990 - val_loss: 1.9943 - val_accuracy: 0.5089
Epo

# قسمت چهارم

<div style="direction:rtl">
در این قسمت از همان مدل قسمت اول استفاده میکنیم. اما تمامی وزن های آن قابل آموزش هستند.

در اینجا سرعت آموزش بسیار بیشتر شده و با همان تعداد epoch قبلی بسیار بهتر آموزش می بیند.

در عین حال سرعت آموزش نسبت به قسمت اول کمتر است چون تعداد بیشتری وزن آموزش می بیند.
</div>

In [23]:
# model completely trainable
model_pretrained = tf.keras.models.load_model('model.h5')

model_config = model_pretrained.get_config()
model_config['layers'][0]['config']['batch_input_shape'] = (None, 32, 32, 3)
model_pretrained = model.__class__.from_config(model_config, custom_objects={})

model_resnet50_full = model_pretrained.layers[-2].output
fc_layer = Dense(10, activation = "softmax")(model_resnet50_full)
model_resnet50_full = tf.keras.models.Model(inputs = model_pretrained.input, outputs = fc_layer)

model_resnet50_full.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

In [24]:
model_resnet50_full.compile(optimizer='adam',loss='categorical_crossentropy',
                      metrics=['accuracy'])
callback = keras.callbacks.EarlyStopping(monitor='accuracy', patience=8)
model_resnet50_full.fit(x_train, y_train, epochs=50, batch_size=2000, validation_data=(x_test, y_test), callbacks=[callback])

Epoch 1/50
25/25 [==============================] - 30s 720ms/step - loss: 2.5972 - accuracy: 0.2588 - val_loss: 31.6687 - val_accuracy: 0.1003
Epoch 2/50
25/25 [==============================] - 17s 667ms/step - loss: 1.6030 - accuracy: 0.4242 - val_loss: 2.2338 - val_accuracy: 0.1515
Epoch 3/50
25/25 [==============================] - 17s 679ms/step - loss: 1.3425 - accuracy: 0.5184 - val_loss: 2.1857 - val_accuracy: 0.1618
Epoch 4/50
25/25 [==============================] - 17s 673ms/step - loss: 1.1369 - accuracy: 0.5974 - val_loss: 2.2285 - val_accuracy: 0.1527
Epoch 5/50
25/25 [==============================] - 16s 652ms/step - loss: 0.9440 - accuracy: 0.6695 - val_loss: 2.2702 - val_accuracy: 0.1675
Epoch 6/50
25/25 [==============================] - 17s 678ms/step - loss: 0.7914 - accuracy: 0.7209 - val_loss: 2.0746 - val_accuracy: 0.2307
Epoch 7/50
25/25 [==============================] - 17s 679ms/step - loss: 0.6798 - accuracy: 0.7599 - val_loss: 2.0354 - val_accuracy: 0.281